## Preliminars

In [ ]:
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import random
import colorsys
from functools import partial
from functools import reduce

filename = 'Iris.csv'
sample_proportions = {'Train': 0.6, 'Validation': 0.2, 'Test': 0.2}
k = 3

data = pd.read_csv(filename, index_col=0)
name_to_index = {name: i for i, name in enumerate(data.iloc[:, -1].unique())}
i_to_cluster = {i: name for name, i in name_to_index.items()}
cluster = data.iloc[:, -1].apply(lambda x: name_to_index[x])
data.drop(data.columns[-1], axis=1, inplace=True)


## Plotting

In [ ]:
def plot_clusters(data, cluster, i_to_name=None, centroid=None):
    plt.rcParams["figure.figsize"] = (8, 8)
    fig, axs = plt.subplots(data.shape[1], data.shape[1],
                            sharex='col', sharey='row', constrained_layout=True)

    if i_to_name is None:
        N = cluster.unique().shape[0]
        i_to_name = [i for i in range(N)]
    else:
        N = len(i_to_name)
    def get_color(n): return colorsys.hsv_to_rgb(n/N, 0.65, 1)

    color_data = cluster.apply(get_color)
    color_cluster = [get_color(i) for i in range(N)]

    for r, r_name in enumerate(data.columns):
        y_data = data[r_name]
        for c, c_name in enumerate(data.columns):
            if r != c:
                axs[r, c].scatter(data[c_name], y_data, c=color_data)
            else:
                axs[r, c].text(0.5, 0.5, r_name,
                               fontweight='bold',
                               horizontalalignment='center',
                               verticalalignment='center',
                               transform=axs[r, c].transAxes)

    if centroid is not None:
        for r, r_name in enumerate(data.columns):
            y_centroid = centroid[r_name]
            for c, c_name in enumerate(data.columns):
                if r != c:
                    axs[r, c].scatter(centroid[c_name], y_centroid, c=color_cluster,
                                      marker=(5, 1), edgecolors='black')

    legend_elements = [Line2D([0], [0], label=i_to_name[i], markerfacecolor=c,
                              color='w', marker='s', markersize=12)
                       for i, c in enumerate(color_cluster)]
    fig.legend(handles=legend_elements, loc='center')
    plt.show()


## Sampling

In [ ]:
def sampling_uniform(data, sample_proportions, cluster=None):
    assert sum(sample_proportions.values()) == 1, \
        sum(sample_proportions.values())

    N = data.shape[0]
    idx = np.random.permutation(data.index)
    data_r = data.reindex(idx)
    cluster_r = None if cluster is None else cluster.reindex(idx)

    p_low = 0
    i_to_name = {}
    data_r['Sample'] = None
    for i, (sample, proportion) in enumerate(sample_proportions.items()):
        p_high = p_low + proportion
        i_low, i_high = int(N * p_low), int(N * p_high)
        data_r.iloc[i_low: i_high, -1] = i
        i_to_name[i] = sample
        p_low = p_high

    return data_r.drop('Sample', axis=1), data_r['Sample'], i_to_name, cluster_r


## Distances

In [ ]:
def dist_euclidean2(p1, p2):
    d = p1 - p2
    return d.dot(d)


def dist_manhattan(p1, p2):
    return abs(p1 - p2).sum()


def dist_mahalanobis2(precision, p1, p2):
    # precision matrix is the inverse of the covariance matrix
    d = p1 - p2
    return d.transpose().dot(precision).dot(d)


def closest(p, points, dist):
    d = points.apply(lambda m: dist(p, m), axis=1)
    return (d.argmin(), d.min())


def normalize(points):
    min_elem = points.min()
    scale = points.max() - min_elem
    def f(x): return (x-min_elem)/scale
    def g(x): return x*scale + min_elem
    return points.apply(f, axis=1), lambda x: x.apply(g, axis=1)



## Clustering

In [ ]:
def get_internal_validation(points, cluster, centroids, dist):
    # Calinski-Harabasz Criterion
    m = points.mean()
    k = centroids.shape[0]
    N = points.shape[0]
    n = centroids.apply(lambda c: (cluster == c.name).sum(), axis=1)
    SSB = centroids.apply(lambda c: n[c.name]*dist(c, m), axis=1).sum()
    SSW = points.apply(lambda p: dist(
        p, centroids.loc[cluster[p.name]]), axis=1).sum()
    VRC = (SSB/SSW) * ((N - k)/(k-1))
    return VRC


def get_external_validation(train_ans, real_ans):
    def paired(t): return t.apply(lambda x: t.apply(lambda y: x == y))

    def triang_inf(t): return t.mask(
        np.triu(np.ones(t.shape)).astype(bool)).stack()

    train_pair = triang_inf(paired(train_ans))
    pair_ans = triang_inf(paired(real_ans))
    TP, FP, FN, TN = 0, 0, 0, 0

    for u, v in zip(train_pair, pair_ans):
        TP += u and v
        FP += u and not v
        FN += not u and v
        TN += not u and not v

    Pr = TP/(TP + FP)
    R = TP/(TP + FN)
    f1_measure = 2*(Pr * R)/(Pr + R)
    return {'Precision': Pr, 'Recall': R, 'F1': f1_measure}


def cluster_kmeans(points, dist, k):
    def get_centroids(points, membership): return pd.DataFrame(
        [points[membership == i].mean() for i in range(k)])

    def assign_cluster(points, centroid, dist):
        c = points.apply(lambda p: pd.Series(
            closest(p, centroid, dist)), axis=1)
        return c[0], c[1].sum()

    membership = points.apply(lambda _: random.randrange(0, k), axis=1)
    centroid = get_centroids(points, membership)
    objective = points.apply(lambda p: dist(
        p, centroid.loc[membership[p.name]]), axis=1).sum()
    delta = objective
    while delta > 0.0001:
        membership_new, objective_new = assign_cluster(points, centroid, dist)
        centroid = get_centroids(points, membership_new)
        delta = abs(objective - objective_new)
        membership, objective = membership_new, objective_new
    return membership, centroid


def cluster_fuzzy_cmeans(points, dist, k, m):
    print(points)
    w = points.apply(lambda x: pd.Series(
        [random.random() for _ in range(k)]), axis=1)
    w = w.apply(lambda wi: wi/wi.sum(), axis=1)
    centroids = pd.DataFrame([
        points.iloc[:, :-1].mul(w[i], axis=0).sum()/w[i].sum() for i in range(k)])
    centroids.iloc[:, -1] = 0
    i, j = 0, 0
    print(centroids.iloc[i, :].apply(lambda cj: 1/centroids.apply(
        lambda ck: dist(cj, ck))))
    print(centroids)
    return True


def train(points, method, dist):
    best_ans, best_centroid = None, None
    best_score = 0
    for i in range(5):
        ans, centroid = method(points, dist)
        score = get_internal_validation(points, ans, centroid, dist)
        if score > best_score:
            best_score, best_ans, best_centroid = score, ans, centroid
    return best_ans, best_centroid


## Run

In [ ]:
data, revert_norm = normalize(data)
#plot_clusters(data, cluster, i_to_cluster)

data_shuffle, data_sample, i_to_sample, cluster_shuffle = sampling_uniform(
    data, sample_proportions, cluster)
#plot_clusters(data_shuffle, data_sample, i_to_sample)

data_train = data_shuffle[data_sample == 0].copy()
cluster_train = cluster_shuffle[data_sample == 0].copy()

ans_train, centroid_train = train(
    data_train.copy(),
    lambda p, dist: cluster_kmeans(p, dist, 3),
    dist_euclidean2)
plot_clusters(revert_norm(data_train), ans_train,
              centroid = revert_norm(centroid_train))

print(get_external_validation(ans_train, cluster_train))
# cluster_fuzzy_cmeans(sample_train.iloc[:,:-2].copy(),
#                     dist_euclidean2,
#                     k, 2)
